In [1]:
import json
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import tkinter as tk
from tkinter import messagebox
from transformers import GPT2LMHeadModel, GPT2Tokenizer

C:\Users\lenovo\anaconda3\envs\chatbottesy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:


# Load the JSON dataset
def load_dataset(file_path):
    with open(file_path, "r") as file:
        dataset = json.load(file)
    return dataset["data"]["qas"]

# Prepare the dataset for pretraining
def prepare_dataset(dataset):
    pretraining_data = ""
    for qa in dataset:
        question = qa["question"]
        answer = qa["answers"]
        pretraining_data += question + "\n" + answer + "\n"
    return pretraining_data

# Define the file path for the dataset
file_path = "C:\\Users\\lenovo\\Desktop\\dataset.json"  # Update with the actual file path

# Load the dataset
dataset = load_dataset(file_path)
pretraining_data = prepare_dataset(dataset)

# Load the pre-trained model and tokenizer
model_name = 'gpt2'  # Change the model name as needed
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set the model to training mode
model.train()

# Tokenize the pretraining data
input_ids = tokenizer.encode(pretraining_data, return_tensors='pt')

# Fine-tune the model on the pretraining data
model.resize_token_embeddings(len(tokenizer))
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

# Set the model back to evaluation mode
model.eval()

# Define a function to generate a response using the local GPT-2 model
def generate_response(user_input):
    input_ids = tokenizer.encode(user_input, return_tensors='pt')
    with torch.no_grad():
        output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Create a GUI window
window = tk.Tk()
window.title("Medical Chatbot")

# Create a text box for user input
first_name_label = tk.Label(window, text="Ask your question and submit or if you do not have any question, write 'bye'")
first_name_label.pack()

user_input_text = tk.Text(window, height=10, width=70)
user_input_text.pack()

# Create a text box for chatbot responses
chatbot_response_text = tk.Text(window, height=10, width=70)
chatbot_response_text.pack()

# Define a function to get user input and generate chatbot response
def get_user_input():
    user_input = user_input_text.get("1.0", tk.END).strip()
    user_input_text.delete("1.0", tk.END)

    if user_input.lower() == "bye":
        messagebox.showinfo("Chatbot", "Goodbye! Take care.")
        window.destroy()
        return

    response = generate_response(user_input)
    chatbot_response_text.insert(tk.END,"Chatbot: " + response + "\n")

# Create a button to submit user input
submit_button = tk.Button(window, text="Submit", command=get_user_input)
submit_button.pack()

# Run the GUI main loop
window.mainloop()

# Calculate and show accuracy and loss
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss.item()
    accuracy = torch.eq(outputs.logits.argmax(dim=-1), input_ids).sum().item() / input_ids.numel()
    print("Accuracy:", accuracy)
    print("Loss:", loss)

# Run in TensorBoard
writer = SummaryWriter()
writer.add_scalar("Accuracy", accuracy)
writer.add_scalar("Loss", loss)
writer.close()

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Accuracy: 0.007352941176470588
Loss: 2.4137604236602783
